In [1]:
import pandas as pd
import numpy as np

## 계층 색인
- 행,열의 각 축에 대해 다중 단계(계층)를 지정하여 데이터에 차원을 설정
- 인덱스에 다차우너 리스트를 전달하면 계층 색인을 지정할 수 있음
- 데이터 구조를 재배열하거나 pivot 테이블과 같은 그룹 기반 작업에 유용
- 재배열 메서드
    - stack(): 컬럼을 로우로 피벗
    - unstack(): 로우를 컬럼으로 피벗

In [2]:
# Series
# 계층 색인 생성시 주의점
# 2단계 게층 색인
# 자료형 : 다차원 리스트 => 아이템 개수 = n 단계
# 상위계층 : 각 계층 별로 하위계층 아이템 개수만큼 작성, 다차원리스트[0]
# 하위계층 : 실제 행 개수와 동일,다차원 리스트[1]

In [13]:
# 실습)
# 상위계층; a b c d
# 하위계층: a(1,2,3) b(1,2) c(1,2,3,4) d(1)
# np.arnage(10)
level_index=[ # 상위계층 인덱스
    ['a','a','a','b','b','c','c','c','c','d'],
    #하위계층 인덱스
    [1,2,3,1,2,1,2,3,4,1]
            ]

s=pd.Series(np.arange(10),index=level_index)
s

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [10]:
# 인덱스 확인
s.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3, 4]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 2, 2, 3], [0, 1, 2, 0, 1, 0, 1, 2, 3, 0]])

In [12]:
# 상위 계층에 접근
s['c']

1    5
2    6
3    7
4    8
dtype: int32

In [14]:
# 계층 색인에 대해 슬라이싱: 라벨 인덱스 규칙(마지막 인덱스도 포함)
s['a':'c']

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
dtype: int32

In [18]:
# 하위 계층에 접근
# 하위 계층이 2인 데이터 조회
# s[상위계층,하위계층]
s[:,2]

a    1
b    4
c    6
dtype: int32

In [19]:
# 하위 계층에 슬라이싱 : loc 메서드
# 상위계층 c , 하위계층 2~4(라벨)(4번도 포함)
s['c'].loc[2:4]

2    6
3    7
4    8
dtype: int32

In [20]:
# 2번 인덱스(range index)부터 4번 인덱스
s['c'][2:4]

3    7
4    8
dtype: int32

In [21]:
# 잘못된 접근 방법
s['c',2:4]

TypeError: ('c', slice(2, 4, None))

In [22]:
s

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [23]:
#unstack() 메서드: Series 의 로우 인덱스를 컬럼으로 변형하여 데이터프레임 생성
# 기본동작: 최하위 계층을 컬럼으로 변형
# 지정 : 인자값으로 계층 레벨을 전달(0:최상위, 1:하위...)
# NaN: 기존 로우 인덱스로 없었던 계층 값
s.unstack()

,1,2,3,4
a,0.0,1.0,2.0,NaN
b,3.0,4.0,NaN,NaN
c,5.0,6.0,7.0,8.0
d,9.0,NaN,NaN,NaN


In [27]:
# 재배열을 적용할 레벨 지정 가능
s.unstack(0)

,a,b,c,d
1,0.0,3.0,5.0,9.0
2,1.0,4.0,6.0,NaN
3,2.0,NaN,7.0,NaN
4,NaN,NaN,8.0,NaN


In [28]:
# stack 메서드 : 컬럼 인덱스를 로우 인덱스로 변형 / 계층 색인(컬럼->하위 계층)
# 데이터프레임->시리즈
# 데이터프레임에 NaN 값이 있던 컬럼, 로우 인덱스 조합은 stack 결과의 계층에서 누락됨
s.unstack(0).stack()

1  a    0.0
   b    3.0
   c    5.0
   d    9.0
2  a    1.0
   b    4.0
   c    6.0
3  a    2.0
   c    7.0
4  c    8.0
dtype: float64

In [29]:
# DataFrame 생성
# 구조: 4*5
# 로우 인덱스 : 상위(2017,2018),하위(상,하)
# 컬럼 인덱스: 상위(서울,경기),하위(서울-강남,잠실,경기-분당,수원,판교)
# np.arange(20)
rows=[[2017,2017,2018,2018],['상','하','상','하']]
cols=[['서울','서울','경기','경기','경기'],['강남','잠실','분당','수원','판교']]

In [31]:
df=pd.DataFrame(np.arange(20).reshape(4,5),index=rows,columns=cols)
df

서울      경기        
        강남  잠실  분당  수원  판교
2017 상   0   1   2   3   4
     하   5   6   7   8   9
2018 상  10  11  12  13  14
     하  15  16  17  18  19

In [33]:
# 컬럼의 상위 게층 접근 : df 일반적인 컬럼 인덱싱 방식
# 서울 데이터-강남
df['서울']['강남']

2017  상     0
      하     5
2018  상    10
      하    15
Name: 강남, dtype: int32

In [34]:
# 방법2. 튜플 타입으로 인덱스로 전달
# 멀티인덱싱[[col1,col2]]
# 계층인덱싱[(lvl1,lvl2)]
df[('서울','강남')]

2017  상     0
      하     5
2018  상    10
      하    15
Name: (서울, 강남), dtype: int32

In [37]:
# 로우의 상위 게층 접근
# df.loc[상위]
# 2017년 데이터
df.loc[2017]  
# 위의 코드와 다른 결과
#df.iloc[0]

서울    경기      
  강남 잠실 분당 수원 판교
상  0  1  2  3  4
하  5  6  7  8  9

In [39]:
# 방법1. 로우 인덱싱을 여러번
# 2017 계층의 '하'계층
df.loc[2017].loc['하']

서울  강남    5
    잠실    6
경기  분당    7
    수원    8
    판교    9
Name: 하, dtype: int32

In [40]:
df.loc[2017].iloc[1]

서울  강남    5
    잠실    6
경기  분당    7
    수원    8
    판교    9
Name: 하, dtype: int32

In [41]:
df.loc[(2017,'하')]

서울  강남    5
    잠실    6
경기  분당    7
    수원    8
    판교    9
Name: (2017, 하), dtype: int32

In [44]:
# 분당~수원까지 조회
df['경기'].loc[:,'분당':'수원']

분당  수원
2017 상   2   3
     하   7   8
2018 상  12  13
     하  17  18

In [45]:
# 로우의 최하위계층이 컬럼의 최하위계층으로 재배열: unstack(기본값)
df.unstack()

서울              경기                    
      강남      잠실      분당      수원      판교    
       상   하   상   하   상   하   상   하   상   하
2017   0   5   1   6   2   7   3   8   4   9
2018  10  15  11  16  12  17  13  18  14  19

In [46]:
# 컬럼의 최하위게층이 로우의 하위계층으로 재배열
# stack을 사용하는데 적합한 경우는 아님
df.stack()

경기    서울
2017 상 강남   NaN   0.0
       분당   2.0   NaN
       수원   3.0   NaN
       잠실   NaN   1.0
       판교   4.0   NaN
     하 강남   NaN   5.0
       분당   7.0   NaN
       수원   8.0   NaN
       잠실   NaN   6.0
       판교   9.0   NaN
2018 상 강남   NaN  10.0
       분당  12.0   NaN
       수원  13.0   NaN
       잠실   NaN  11.0
       판교  14.0   NaN
     하 강남   NaN  15.0
       분당  17.0   NaN
       수원  18.0   NaN
       잠실   NaN  16.0
       판교  19.0   NaN

In [47]:
# 원상복구
df.unstack().stack()

경기          서울    
        분당  수원  판교  강남  잠실
2017 상   2   3   4   0   1
     하   7   8   9   5   6
2018 상  12  13  14  10  11
     하  17  18  19  15  16

In [50]:
# 계층의 인덱스 번호 또는 라벨을 사용하여 계층의 순서를 변경
# swaplevel(lvl1,lvl0,axis=0(기본값:로우인덱스에 접근))
df.swaplevel(1,0,axis=0)

서울      경기        
        강남  잠실  분당  수원  판교
상 2017   0   1   2   3   4
하 2017   5   6   7   8   9
상 2018  10  11  12  13  14
하 2018  15  16  17  18  19

## 객체 복사
- 할당기호/ 슬라이싱 : 원본과 상호 종속적인 복사본 객체 생성
- obj.copy(): 원본과 독립적인 복사본 객체 생성

### 정렬
- obj.sort_index(): 인덱스를 기준으로 정렬(기본값은 ascending=True,오름차순 정렬)
    - DataFrame
        - axis = 0 : 기본값, 로우 인덱스 기준으로 정렬
        - axis=1: 컬럼 인덱스 기준으로 정렬
- obj.sort_values(): 값을 기준으로 정렬
    - DataFrame
        - by: 정렬의 기준이 되는 인덱스 값 전달
        - axis=0: 기본값, 컬럼을 기준으로 로우 인덱스를 정렬하며 기준값으로 by에 인덱스 컬럼 레벨 또는 컬럼명 전달
        - axis=1: 로우 인덱스를 기준으로 컬럼 라벨을 정렬하며 기준값으로 by에 레벨 도는 라벨명 전달

In [61]:
# Series 생성
# 값과 인덱스 라벨이 순서대로 들어가지 않은 series
# 값: 2,1,5,10,3
# 인덱스 라벨: atbgc
s1=pd.Series([2,1,5,10,3],index=list('atbgc'))
s1

a     2
t     1
b     5
g    10
c     3
dtype: int64

In [62]:
# 인덱스 기준 오름차순으로 정렬
# 기본동작 : 오름차순, 기본축=행/로우 인덱스 정렬
s1.sort_index()

a     2
b     5
c     3
g    10
t     1
dtype: int64

In [63]:
# 내림차순
s1.sort_index(ascending=False)

t     1
g    10
c     3
b     5
a     2
dtype: int64

In [64]:
# 값 기준 오름차순으로 정렬
s1.sort_values()

t     1
a     2
c     3
b     5
g    10
dtype: int64

In [65]:
s1.sort_values(ascending=False)

g    10
b     5
c     3
a     2
t     1
dtype: int64

In [66]:
# 구조 : 4*5
# randint(20)
# 로우 인덱스 : hcae
# 컬럼 인덱스 : EAFCD
df=pd.DataFrame(np.random.randint(20,size=(4,5)),index=list('hcae'),columns=list('EAFCD'))
df

,E,A,F,C,D
h,19,2,15,13,10
c,3,9,8,5,5
a,3,0,11,18,0
e,6,0,8,5,2


In [67]:
# 로우 인덱스 오름차순
df.sort_index()

,E,A,F,C,D
a,3,0,11,18,0
c,3,9,8,5,5
e,6,0,8,5,2
h,19,2,15,13,10


In [68]:
# 내림차순
df.sort_index(ascending=False)

,E,A,F,C,D
h,19,2,15,13,10
e,6,0,8,5,2
c,3,9,8,5,5
a,3,0,11,18,0


In [69]:
# 컬럼기준
df.sort_index(axis=1)

,A,C,D,E,F
h,2,13,10,19,15
c,9,5,5,3,8
a,0,18,0,3,11
e,0,5,2,6,8


In [70]:
# 컬럼 기준으로 내림차순 정렬 후 로우 기준으로 오름차순 정렬
df.sort_index(axis=1,ascending=False).sort_index()

,F,E,D,C,A
a,11,3,0,18,0
c,8,3,5,5,9
e,8,6,2,5,0
h,15,19,10,13,2


In [71]:
# 원본 바뀌지 않음
df

,E,A,F,C,D
h,19,2,15,13,10
c,3,9,8,5,5
a,3,0,11,18,0
e,6,0,8,5,2


In [72]:
# 값 기준 정렬
# df.sort_values()
# axis=0 col
# axis=1 row
# 기본 동작(axis=0): 컬럼명 입력
# axis=1 : 로우인덱스 입력

In [73]:
# 컬럼 D의 값을 내림차순으로 정렬
# 결과: 로우 인덱스가 섞임/ 행이 섞임
df.sort_values('D',ascending=False)

,E,A,F,C,D
h,19,2,15,13,10
c,3,9,8,5,5
e,6,0,8,5,2
a,3,0,11,18,0


In [74]:
# 인덱스 라벨 'a'의 값을 기준으로 오름차순 정렬
# 결과 : 열이 섞임
df.sort_values('a',axis=1)

,A,D,E,F,C
h,2,10,19,15,13
c,9,5,3,8,5
a,0,0,3,11,18
e,0,2,6,8,5


In [75]:
# 두개의 컬럼에 대해 정렬: 정렬 우선순위를 컬럼에 적용
# 첫번째 컬럼을 기준으로 우선 정렬
# 첫번째 컬럼에서 동일한 값이 있을 때 두번째 컬럼 값에 대해 재졍렬
# 정렬값 : 리스트로 묶어서 전달
# 컬럼 E와 A의 값을 기준으로 오름차순 정렬
# 컬럼 E -> 동일값이 있을 경우에만 -> A에서 재정렬
df.sort_values(['E','A'])

,E,A,F,C,D
a,3,0,11,18,0
c,3,9,8,5,5
e,6,0,8,5,2
h,19,2,15,13,10


In [76]:
# 두개의 컬럼에 대해 서로 다른 정렬
# ascending = [] => 각 칼럼에 적용할 정렬 순서
# E 오름 A 내림
df.sort_values(['E','A'],ascending=[True,False])

,E,A,F,C,D
c,3,9,8,5,5
a,3,0,11,18,0
e,6,0,8,5,2
h,19,2,15,13,10


### 연습문제

In [108]:
col=[[2016,2016,2017,2017],['영어','수학','영어','수학']]
row=['Kim','Park','Lee','Jung','Moon']
score=pd.DataFrame(np.random.randint(50,100,size=(5,4)),columns=col,index=row)
#name 속성
score.index.name='학생명'
score.columns.names='년도','과목'
score

년도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    66  88   94  90
Park   94  82   93  94
Lee    69  78   89  78
Jung   77  88   72  88
Moon   55  78   73  67

In [ ]:
# set_names() 메서드
# score.index.set_names('학생명',inplace=True)
# score.columns.set_names(['연도2','과목2'],inplace=True)

In [111]:
# 2016년 데이터만 별도 저장
df2016=score[2016].copy()
df2016

과목,영어,수학
학생명,,
Kim,66,88
Park,94,82
Lee,69,78
Jung,77,88
Moon,55,78


In [112]:
# 학생 이름 기준으로 오름차순 정렬
df2016.sort_index()

과목,영어,수학
학생명,,
Jung,77,88
Kim,66,88
Lee,69,78
Moon,55,78
Park,94,82


In [123]:
# 영어 점수가 높은 순서로 정렬하고, 영어 점수 동점자의 경우 수학 점수가 낮은 순서로 정렬
df2016.loc['Jung']=[66,98]
df2016

과목,영어,수학
학생명,,
Kim,66,88
Park,94,82
Lee,69,78
Jung,66,98
Moon,55,78


In [142]:
df2016.sort_values(['영어','수학'],ascending=['False','True'])

과목,영어,수학
학생명,,
Moon,55,78
Kim,66,88
Jung,66,98
Lee,69,78
Park,94,82


In [150]:
# 2016년, 2017년 데이터 모두 저장하고 있는 데이터프레임에 대해 2017년 수학 성적 기준으로 모든 값을 내림차순 정렬
score.sort_values((2017,'수학'),ascending=False)

년도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Park   94  82   93  94
Kim    66  88   94  90
Jung   77  88   72  88
Lee    69  78   89  78
Moon   55  78   73  67

In [151]:
# 2017년 수학점수 내림차순
# 동점자 : 2016년 수학점수
score[(2017,'수학')]['Jung']=98

In [152]:
score.sort_values([(2017,'수학'),(2016,'수학')],ascending=[False,True])

년도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Jung   77  88   72  98
Park   94  82   93  94
Kim    66  88   94  90
Lee    69  78   89  78
Moon   55  78   73  67

## 데이터 합치기
- merge
    - pandas 객체의 메서드로 pandas.merge(df1,df2...)로 사용
    - 두 개의 데이터프레임에 대해 특정 컬럼을 기준으로 합치기
    - 주요 파라미터
        - how : 합치는 방식으로 inner(기본값),left,right,outer 방식 존재
        - on : 합치는 기준으로 두 개의 데이터프레임에 공통으로 존재하는 컬럼명을 사용해야함(기본값 = None)

In [4]:
# 고객번호와 고객명을 저장하고 있는 데이터프레임 생성
# 데이터: 딕셔너리 타입
# 구조; 5*2
df1= pd.DataFrame({'no':[30,31,32,33,34],'name':['박나래','전현무','한혜진','헨리','성훈']})
df1

,no,name
0,30,박나래
1,31,전현무
2,32,한혜진
3,33,헨리
4,34,성훈


In [5]:
# 고객번호와 주문수량을 저장하고 있는 데이터프레임 생성
# 다른 고객번호(no)를 일부 지정
# 구조: 4*2
df2=pd.DataFrame({'no':[30,32,40,41],'amount':[100,130,20,50]})
df2

,no,amount
0,30,100
1,32,130
2,40,20
3,41,50


In [6]:
# 가장 기본적인 merge 방식
# how='inner': 교집합 => 두 데이터프레임의 공통 컬럼의 값이 가지는 교집합
# df1 & df2 공통컬럼 : no
# 공통컬럼 값에서의 교집합 : 30,32
# df1 30~34
# df2 30,31,40,41
pd.merge(df1,df2)

,no,name,amount
0,30,박나래,100
1,32,한혜진,130


In [7]:
# outer : 공통 컬럼 값에서의 합집합
pd.merge(df1,df2, how='outer')

,no,name,amount
0,30,박나래,100.0
1,31,전현무,NaN
2,32,한혜진,130.0
3,33,헨리,NaN
4,34,성훈,NaN
5,40,NaN,20.0
6,41,NaN,50.0


In [8]:
# left : 첫번째 데이터프레임(df1)의 값(공통컬럼 값 기준)을 모두 보존
# df2에서 가져오는 값 : 공통컬럼에서 df1에 값이 있는 것만 가져옴
pd.merge(df1,df2,how='left')

,no,name,amount
0,30,박나래,100.0
1,31,전현무,NaN
2,32,한혜진,130.0
3,33,헨리,NaN
4,34,성훈,NaN


In [9]:
# right : 두번째 데이터프레임(df2)의 값(공통컬럼 값 기준)을 모두 보존
pd.merge(df1,df2,how='right')

,no,name,amount
0,30,박나래,100
1,32,한혜진,130
2,40,NaN,20
3,41,NaN,50


In [15]:
# 공통 컬럼이 두개 이상인 경우
# 고객명, 날짜, 정보를 저장하고 있는 데이터프레임 생성
df3=pd.DataFrame({'name':['박나래','성훈','헨리'],
                 'date':['2017-01-05','2017-02-02','2017-01-02'],
                 'info':['010','011','017']})
df3

,name,date,info
0,박나래,2017-01-05,010
1,성훈,2017-02-02,011
2,헨리,2017-01-02,017


In [13]:
df4=pd.DataFrame({'name':['박나래','헨리','이시언'],
                 'info':['F','M','M']})
df4

,name,info
0,박나래,F
1,헨리,M
2,이시언,M


In [16]:
# 공통 컬럼이 두 개 이상인 경우에 결과값 안나옴
pd.merge(df3,df4)

,name,date,info


In [13]:
# on 파라미터: 공통 컬ㄹ머이 두 개 이상 존재할 때 특정 하나의 컬럼을 key로 지정
pd.merge(df3,df4,on='name')

,name,date,info_x,info_y
0,박나래,2017-01-05,010,F
1,헨리,2017-01-02,017,M


In [18]:
# left_on,rigth_on: 결합하려는 key지정
df5=pd.DataFrame({'c_name':['박나래','헨리','이시언'],
                 'info':['F','M','M']})

In [20]:
tmp=pd.merge(df3,df5,left_on='name',right_on='c_name')

In [21]:
# 불필요한 컬럼 정리: df.drop(컬럼명,axis=1[컬럼축],inplace=True[원본 바로 반영])
tmp.drop('c_name',axis=1,inplace=True)
tmp

,name,date,info_x,info_y
0,박나래,2017-01-05,010,F
1,헨리,2017-01-02,017,M


### concat
- 특정 key를 기준으로 데이터를 합치는 것이 아니라 행,열 기준으로 데이터를 연결
- 주요 파라미터
    - axis:0 / 행 방향(기본값)이며 컬럼을 key로 합치고, 1/ 열 방향으로 로우를 key로 합침
    - join: 데이터프레임끼리 연결할 때 합치는 방법으로 outer(기본값),inner 방식 존재
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [3]:
# 공통 인덱스 라벨을 가지는 series 2개 생성
# 3개 abc
# 5개 abef
# 공통 인덱스: ab
s1=pd.Series([1,2,3],index=list('abc'))
s2=pd.Series([1,2,3,4],index=list('abef'))

In [4]:
s1

a    1
b    2
c    3
dtype: int64

In [5]:
s2

a    1
b    2
e    3
f    4
dtype: int64

In [6]:
# 두개의 시리즈 끼리 연결
# pd.concat([obj,obj...]): 행 방향으로 연결
# 순서대로 아래로 연결
# 인덱스 라벨은 기존값 유지
pd.concat([s1,s2])

a    1
b    2
c    3
a    1
b    2
e    3
f    4
dtype: int64

In [7]:
# 새로운 인덱스 생성
pd.concat([s1,s1],ignore_index=True)

0    1
1    2
2    3
3    1
4    2
5    3
dtype: int64

In [8]:
# 열 방향 연결 => sr 끼리의 결합으로 df 생성
# 행 길이가 다른 경우
# 라벨 인덱스도 다른 경우
# 결과 해석: 결과 df의 라벨 인덱스 = s1,s2 라벨 인덱스의 합집합
# 원본 sr에 없느 라벨 인덱스 => NaN
pd.concat([s1,s2],axis=1,sort=False)  # sort 빨간거 안보이게 하기 위한 것(필수 아님)

,0,1
a,1.0,1.0
b,2.0,2.0
c,3.0,NaN
e,NaN,3.0
f,NaN,4.0


In [9]:
# 열 이름 설정: keys 파라미터 인자값으로 열이름 리스트를 전달
# 다른방법 ) df의 컬럼 속성에 열이름 리스트 전달 / df.columns=['col1','col2']
pd.concat([s1,s2],axis=1,sort=False,keys=['col1','col2'])

,col1,col2
a,1.0,1.0
b,2.0,2.0
c,3.0,NaN
e,NaN,3.0
f,NaN,4.0


In [26]:
# 두개의 데이터 프레임 생성 및 연결
# 고객명, 날짜 구매금액 : df
# 고객명, 성별 : df4
df4

,name,info
0,박나래,F
1,헨리,M
2,이시언,M


In [10]:
df=pd.DataFrame({'name':['박나래','이시언','한혜진','성훈'],
                'date':['2019','2018','2020','1991'],
                'amount':[10,20,30,40]})

In [11]:
df

,name,date,amount
0,박나래,2019,10
1,이시언,2018,20
2,한혜진,2020,30
3,성훈,1991,40


In [17]:
pd.concat([df,df4],sort=False)

,name,date,amount,info
0,박나래,2019,10.0,NaN
1,이시언,2018,20.0,NaN
2,한혜진,2020,30.0,NaN
3,성훈,1991,40.0,NaN
0,박나래,NaN,NaN,F
1,헨리,NaN,NaN,M
2,이시언,NaN,NaN,M


In [18]:
# 열 방향으로 데이터프레임 연결
pd.concat([df,df4],axis=1)

,name,date,amount,name,info
0,박나래,2019,10,박나래,F
1,이시언,2018,20,헨리,M
2,한혜진,2020,30,이시언,M
3,성훈,1991,40,NaN,NaN


In [19]:
# 공통 컬럼이 없는 데이터프레임 생성
# df,df4
df5=pd.DataFrame({'city':['서울','부산'],
                 'job':['A','B']})
df5

,city,job
0,서울,A
1,부산,B


In [20]:
pd.concat([df,df5],axis=1)

,name,date,amount,city,job
0,박나래,2019,10,서울,A
1,이시언,2018,20,부산,B
2,한혜진,2020,30,NaN,NaN
3,성훈,1991,40,NaN,NaN


### append
- 행 방향으로 데이터를 연결
- 주요 파라미터
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [21]:
# 두개의 시리즈 생성
s1=pd.Series([1,2,3])
s2=pd.Series([1,2,3,4])

In [22]:
s1

0    1
1    2
2    3
dtype: int64

In [24]:
# 행 방향으로 결합하고 기존의 인덱스 유지
s1.append(s2)
# pd.concat([s1,s2])

0    1
1    2
2    3
0    1
1    2
2    3
3    4
dtype: int64

In [25]:
# 행 방향으로 데이터 연결하고 새로운 인덱스 지정
s1.append(s2,ignore_index=True)

0    1
1    2
2    3
3    1
4    2
5    3
6    4
dtype: int64

In [26]:
# 두 개의 데이터프레임
# 서로 다른 컬럼명 , 서로 다른 로우 개수
# df1: 1,2,3,4 / col a,b
# df2 : 5,6,7,8,9,10 / col A,B
df1 = pd.DataFrame([[1,2],[3,4]], columns=list('ab'))
df2 = pd.DataFrame([[5,6],[7,8],[9,10]], columns=list('AB'))

In [28]:
df2

,A,B
0,5,6
1,7,8
2,9,10


In [30]:
df1.append(df2,sort=False)

,a,b,A,B
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
0,NaN,NaN,5.0,6.0
1,NaN,NaN,7.0,8.0
2,NaN,NaN,9.0,10.0


In [31]:
# 동일한 컬럼명을 가진 두개의 데이터프레임
df3=pd.DataFrame([[10,20],[30,40]],columns=list('ab'))

In [32]:
df3

,a,b
0,10,20
1,30,40


In [33]:
# 동일한 컬럼명에 대한 append 
# 컬럼이 추가되지 않고 로우만 아래로 확장되는 결과
# append가 가장 많이 사용되는 / 편하게 쓸 수 있는 용도
df1.append(df3, ignore_index=True)

,a,b
0,1,2
1,3,4
2,10,20
3,30,40


In [39]:
# 시리즈와 데이터프레임 
# 라벨 인덱스: ab
s1=pd.Series([11,12],index=list('ab'))
s1

a    11
b    12
dtype: int64

In [41]:
s1.name='A'
s1

a    11
b    12
Name: A, dtype: int64

In [42]:
# df과 s1를 append 할 때는 ignore_index=True 설정
# 또는 s1의 name 속성이 있을 때는 name 값이 라벨인덱스로 적용
# sr name 필수
df1.append(s1)

,a,b
0,1,2
1,3,4
A,11,12


In [43]:
s2=pd.Series([33,33,33],index=list('acd'))
s2

a    33
c    33
d    33
dtype: int64

In [45]:
df1.append(s2,ignore_index=True)

,a,b,c,d
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
2,33.0,NaN,33.0,33.0


### 집계

- groupby(컬럼명)
    - 특정 속성을 기준으로 묶어서 다양한 집계 함수 적용
    - 대표적인 집계 함수
        - sum : 총합
        - mean : 평균값
        - min : 최소값
        - max : 최대값
        - count : 개수
        - std : 표준편차
- pivot table
    - df.pivot_table(로우로 사용될 컬럼명, 컬럼으로 사용될 컬럼명, 튜플을 구성하는 값으로 사용될 컬럼명, 집계함수)
    - 일차원으로 컬럼 및 로우가 단순 나열된 형식은 데이터를 파악하는데 적합하지 않기 때문에 pivot을 통해 계층 색인 및 형태 변경을 수행

In [46]:
# 엑셀 데이터 불러오기
data=pd.read_excel('인구수예제.xlsx')

In [68]:
data

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


In [48]:
# 데이터 구조 확인
data.shape

(50, 6)

In [49]:
# 데이터 상위 5개
data.head()

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190


In [51]:
# 데이터 하위 5개
data.tail()

,도시,자치구,연도,남자인구,여자인구,총인구
45,부산,동래구,2013,83,65,148
46,부산,동래구,2014,139,87,226
47,부산,동래구,2015,147,115,262
48,부산,동래구,2016,61,102,163
49,부산,동래구,2017,132,105,237


In [53]:
# 데이터중 랜덤 추출
data.sample(10)

,도시,자치구,연도,남자인구,여자인구,총인구
34,서울,동작구,2017,79,134,213
35,부산,해운대구,2013,124,103,227
2,서울,강남구,2015,123,83,206
48,부산,동래구,2016,61,102,163
45,부산,동래구,2013,83,65,148
23,서울,송파구,2016,80,92,172
6,서울,서대문구,2014,149,150,299
36,부산,해운대구,2014,101,144,245
7,서울,서대문구,2015,106,77,183
32,서울,동작구,2015,74,139,213


#### group by(속성명)+집계함수

In [54]:
data['자치구'].unique()

array(['강남구', '서대문구', '종로구', '영등포구', '송파구', '도봉구', '동작구', '해운대구', '수영구',
       '동래구'], dtype=object)

In [55]:
# 자치구별 남, 여 인구 각각의 총합
# groupby 기준으로 사용된 컬럼 => 로우 인덱스로 적용
data.groupby('자치구')[['남자인구', '여자인구']].sum()

,남자인구,여자인구
자치구,,
강남구,539,513
도봉구,485,550
동래구,562,474
동작구,454,582
서대문구,488,543
송파구,415,559
수영구,502,559
영등포구,629,562
종로구,483,373


In [61]:
# 도시별&자치구별&남여인구 각각 총합
tmp=data.groupby(['도시','자치구'])[['남자인구','여자인구']].sum()

In [64]:
tmp

남자인구  여자인구
도시 자치구             
부산 동래구    562   474
   수영구    502   559
   해운대구   620   515
서울 강남구    539   513
   도봉구    485   550
   동작구    454   582
   서대문구   488   543
   송파구    415   559
   영등포구   629   562
   종로구    483   373

In [65]:
tmp.index

MultiIndex(levels=[['부산', '서울'], ['강남구', '도봉구', '동래구', '동작구', '서대문구', '송파구', '수영구', '영등포구', '종로구', '해운대구']],
           labels=[[0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [2, 6, 9, 0, 1, 3, 4, 5, 7, 8]],
           names=['도시', '자치구'])

In [66]:
data.head(1)

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165


In [67]:
# 도시별 연도별 총인구 평균
data.groupby(['도시','연도'])['총인구'].mean()

도시  연도  
부산  2013    201.000000
    2014    227.666667
    2015    199.000000
    2016    217.333333
    2017    232.333333
서울  2013    218.142857
    2014    224.714286
    2015    178.714286
    2016    199.000000
    2017    204.428571
Name: 총인구, dtype: float64

In [70]:
# 데이터파악에 용이한 구조로 변경
data.groupby(['도시','연도'])['총인구'].mean().unstack()

연도,2013,2014,2015,2016,2017
도시,,,,,
부산,201.000000,227.666667,199.000000,217.333333,232.333333
서울,218.142857,224.714286,178.714286,199.000000,204.428571


In [71]:
# 데이터파악에 용이한 구조로 변경2
data.groupby(['도시','연도'])['총인구'].mean().unstack(0)

도시,부산,서울
연도,,
2013,201.000000,218.142857
2014,227.666667,224.714286
2015,199.000000,178.714286
2016,217.333333,199.000000
2017,232.333333,204.428571


#### groupby 결과를 pivot_table 으로 나타낼 수 있다.

In [72]:
# 연도별 & 도시별 & 남여 인구 각각의 최대값
# df.pivot_table(index=로우인덱스로 사용할 컬럼, columns=컬럼으로 사용할 컬럼,
#               values=집계함수를 적용할 컬럼, aggfunc='mean'/집계함수)
# 로우 : 연도, 도시
# 값 : 남자인구, 여자인구
# 집계함수 : 최대값
# data.pivot_table(index=['연도','도시'], columns='자치구', values=['남자인구','여자인구'],
#                  aggfunc='max')
data.pivot_table(index=['연도','도시'], values=['남자인구','여자인구'],
                 aggfunc='max')

남자인구  여자인구
연도   도시            
2013 부산   134   103
     서울   146   138
2014 부산   139   144
     서울   149   150
2015 부산   147   115
     서울   123   139
2016 부산   134   148
     서울   147   150
2017 부산   146   105
     서울   145   150

In [80]:
# 도시별 연도별 총인구 평균
data.groupby(['도시','연도'])['총인구'].mean().unstack(0)

도시,부산,서울
연도,,
2013,201.000000,218.142857
2014,227.666667,224.714286
2015,199.000000,178.714286
2016,217.333333,199.000000
2017,232.333333,204.428571


In [75]:
data.pivot_table(index='연도',columns='도시',values='총인구',aggfunc='mean')

도시,부산,서울
연도,,
2013,201.000000,218.142857
2014,227.666667,224.714286
2015,199.000000,178.714286
2016,217.333333,199.000000
2017,232.333333,204.428571


In [76]:
df4

,name,info
0,박나래,F
1,헨리,M
2,이시언,M


In [81]:
# 각각의 컬럼에 서로 다른 집계함수 적용 : 딕셔너리 타입으로 aggfunc 인자값 전달
# 도시별 자치구별 남자인구 평균, 여자인구 총합
# 로우 : 도시, 자치구
# 값 : 남자, 여자
# 집계함수 : 평균, 총합
tmp=data.pivot_table(index=['도시','자치구'],values=['남자인구','여자인구'],aggfunc={'남자인구':'mean','여자인구':'sum'})

In [82]:
tmp

남자인구  여자인구
도시 자치구              
부산 동래구   112.4   474
   수영구   100.4   559
   해운대구  124.0   515
서울 강남구   107.8   513
   도봉구    97.0   550
   동작구    90.8   582
   서대문구   97.6   543
   송파구    83.0   559
   영등포구  125.8   562
   종로구    96.6   373

In [83]:
tmp.columns=['남자 평균 인구','여자 총합 인구']
tmp

남자 평균 인구  여자 총합 인구
도시 자치구                     
부산 동래구      112.4       474
   수영구      100.4       559
   해운대구     124.0       515
서울 강남구      107.8       513
   도봉구       97.0       550
   동작구       90.8       582
   서대문구      97.6       543
   송파구       83.0       559
   영등포구     125.8       562
   종로구       96.6       373

In [84]:
# 하나의 값에 여러 가지 집계함수 적용 => 집계함수 개수만큼 컬럼 생성 
data.pivot_table(index=['도시','자치구'], values='남자인구',
                aggfunc=['mean','sum']).swaplevel(1,0,axis=1)

남자인구     
          mean  sum
도시 자치구             
부산 동래구   112.4  562
   수영구   100.4  502
   해운대구  124.0  620
서울 강남구   107.8  539
   도봉구    97.0  485
   동작구    90.8  454
   서대문구   97.6  488
   송파구    83.0  415
   영등포구  125.8  629
   종로구    96.6  483

#### pivot_table 실습

In [85]:
# 1. 도시별, 자치구별 남자인구 평균
# 로우 : 도시, 자치구
# 값 : 남자인구
# 함수 : 평균
tmp1 = data.pivot_table(index=['도시', '자치구'], values='남자인구', aggfunc='mean')

In [86]:
tmp1

남자인구
도시 자치구        
부산 동래구   112.4
   수영구   100.4
   해운대구  124.0
서울 강남구   107.8
   도봉구    97.0
   동작구    90.8
   서대문구   97.6
   송파구    83.0
   영등포구  125.8
   종로구    96.6

In [90]:
# 2. 도시별, 자치구별 평균 남자인구 상위(많은 순서) 5개 자치구 조회
# 값 정렬 : 내림차순
# 상위 5개
# head().index.get_level_values(1)
tmp1.sort_values('남자인구',ascending=False).head()

남자인구
도시 자치구        
서울 영등포구  125.8
부산 해운대구  124.0
   동래구   112.4
서울 강남구   107.8
부산 수영구   100.4

In [ ]:
# 3. 도시별 자치구별 5년치 평균 인구(남자/여자 각각)에 대해 
# 남자인구와 여자인구의 차이가 가장 큰 자치구 상위 5개 
# 참고 함수 : abs()
# '평균차이' 컬럼 추가

In [91]:
tmp2=data.pivot_table(index=['도시','자치구'],values=['남자인구','여자인구'],aggfunc='mean')
tmp2

남자인구   여자인구
도시 자치구               
부산 동래구   112.4   94.8
   수영구   100.4  111.8
   해운대구  124.0  103.0
서울 강남구   107.8  102.6
   도봉구    97.0  110.0
   동작구    90.8  116.4
   서대문구   97.6  108.6
   송파구    83.0  111.8
   영등포구  125.8  112.4
   종로구    96.6   74.6

In [92]:
tmp2['평균차이']=abs(tmp2['남자인구']-tmp2['여자인구'])

In [93]:
tmp2

남자인구   여자인구  평균차이
도시 자치구                     
부산 동래구   112.4   94.8  17.6
   수영구   100.4  111.8  11.4
   해운대구  124.0  103.0  21.0
서울 강남구   107.8  102.6   5.2
   도봉구    97.0  110.0  13.0
   동작구    90.8  116.4  25.6
   서대문구   97.6  108.6  11.0
   송파구    83.0  111.8  28.8
   영등포구  125.8  112.4  13.4
   종로구    96.6   74.6  22.0

In [95]:
# 평균차이가 가장 큰 자치구 5개
tmp2.sort_values('평균차이',ascending=False).head()

남자인구   여자인구  평균차이
도시 자치구                     
서울 송파구    83.0  111.8  28.8
   동작구    90.8  116.4  25.6
   종로구    96.6   74.6  22.0
부산 해운대구  124.0  103.0  21.0
   동래구   112.4   94.8  17.6

In [97]:
s1=abs(tmp2['남자인구']-tmp2['여자인구'])

In [99]:
s1.name='평균차이'

In [101]:
# 평균차이 series를 concat 으로 df의 컬럼으로 추가
# series.name 속성 -> df 컬럼명으로 적용
pd.concat([tmp2,s1],axis=1)

남자인구   여자인구  평균차이  평균차이
도시 자치구                           
부산 동래구   112.4   94.8  17.6  17.6
   수영구   100.4  111.8  11.4  11.4
   해운대구  124.0  103.0  21.0  21.0
서울 강남구   107.8  102.6   5.2   5.2
   도봉구    97.0  110.0  13.0  13.0
   동작구    90.8  116.4  25.6  25.6
   서대문구   97.6  108.6  11.0  11.0
   송파구    83.0  111.8  28.8  28.8
   영등포구  125.8  112.4  13.4  13.4
   종로구    96.6   74.6  22.0  22.0

### 통계

- 주요 통계 함수
    - value_counts() : 각 고유값의 중복개수
    - count : NaN 값을 제외한 값의 개수
    - describe : 각 컬럼에 대한 요약통계 계산(count, mean, std, min, 1사분위수, 중위값, 3사분위수, max)
    - min, max : 최소, 최대 값
    - sum : 총 합
    - cumsum : 누적합
    - mean : 평균
    - median : 중위값(전체 데이터를 나열 했을 때 중간에 위치한 값)
    - var : 분산(데이터가 전체적으로 흩어진 정도, 편차제곱의 평균)
    - std : 표준편차 = 분산의 양의 제곱근
- 주요 파라미터
    - axis : 연산의 기준이 되는 축, axis=0(기본값)이면 행 방향으로 axis=1이면 열 방향으로 적용
    - skipna : NaN 값을 제외할지 여부를 설정, 기본값 = True

In [103]:
import numpy as np

In [104]:
# 샘플데이터 생성
df = pd.DataFrame(np.random.randint(50, 100, (5,4)),
                 index = ['kim', 'park', 'lee', 'jung', 'moon'],
                 columns = [[2016, 2016, 2017, 2017],
                            ['영어', '수학', '영어', '수학']])
df.index.set_names('학생명', inplace=True)
df.columns.set_names(['연도', '과목'], inplace=True)
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
kim    59  68   66  65
park   88  71   51  93
lee    52  83   84  96
jung   99  86   55  56
moon   51  99   55  88

In [107]:
# 기술통계, 요약통계 : 기본동작 => 수치형 데이터에 대해서만 반환
df.describe()

연도          2016                  2017           
과목            영어         수학         영어         수학
count   5.000000   5.000000   5.000000   5.000000
mean   69.800000  81.400000  62.200000  79.600000
std    22.196847  12.461942  13.405223  17.952716
min    51.000000  68.000000  51.000000  56.000000
25%    52.000000  71.000000  55.000000  65.000000
50%    59.000000  83.000000  55.000000  88.000000
75%    88.000000  86.000000  66.000000  93.000000
max    99.000000  99.000000  84.000000  96.000000

In [108]:
# 2017년 데이터에 대해서만 분리해서 저장
df2017=df[2017].copy()
df2017

과목,영어,수학
학생명,,
kim,66,65
park,51,93
lee,84,96
jung,55,56
moon,55,88


In [109]:
# 1) count() : NaN 값을 제외한 데이터 개수(기본:행 개수)
# axis=1 (열 개수)
df2017.count()

과목
영어    5
수학    5
dtype: int64

In [111]:
df2017.count(axis=1)

학생명
kim     2
park    2
lee     2
jung    2
moon    2
dtype: int64

In [112]:
df2017.shape

(5, 2)

In [113]:
# value_counts() : 범주형 데이터에 대해서 각 범주의 데이터 개수를 반환
data['도시'].value_counts()

서울    35
부산    15
Name: 도시, dtype: int64

In [114]:
# 2) sum() : 총합
# axis=0(기본) : 행의 합 => 모든 학생의 과목별 총합 => 결과는 열단위
df2017.sum()

과목
영어    311
수학    398
dtype: int64

In [115]:
# axis=1 => 열의 합 => 각 학생별로 과목 총합 => 결과 : 행 단위
df2017.sum(axis=1)

학생명
kim     131
park    144
lee     180
jung    111
moon    143
dtype: int64

In [116]:
# 3) mean() 
# 과목별 모든 학생 점수에 대해 평균
df2017.mean()

과목
영어    62.2
수학    79.6
dtype: float64

In [117]:
# 학생별 두 과목 평균
df2017.mean(axis=1)

학생명
kim     65.5
park    72.0
lee     90.0
jung    55.5
moon    71.5
dtype: float64

In [120]:
# 실습1 : 2016, 2017를 저장하고 있는 df 대상으로 연도별 / 과목별 평균 성적을 출력
df.mean()

연도    과목
2016  영어    69.8
      수학    81.4
2017  영어    62.2
      수학    79.6
dtype: float64

In [121]:
# 실습2 : 2016, 2017를 저장하고 있는 df 대상으로 학생별로 모든 연도/과목에 대한 전체 평균을 출력
df.mean(axis=1)

학생명
kim     64.50
park    75.75
lee     78.75
jung    74.00
moon    73.25
dtype: float64